In [4]:
pip install redis geopandas folium  geopy --index-url https://pypi.org/simple


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import redis
import random
import time
import geopandas as gpd
import folium
from shapely.geometry import Point
import geopy.distance

In [21]:
# Control levers for simulation // Adjust here

NUM_OF_DRIVERS = 10
NUM_OF_RIDERS = 10
RADIUS = 1 # kms

In [22]:
# Start Redis

# Connect to Redis 
redis_client = redis.Redis(host='localhost', port=6379)


In [23]:
# Let's do a clean up 
redis_client.delete("available_drivers")
redis_client.delete("rider_queries")

0

In [24]:
# Function for simulating driver check-ins in Delhi area (adjust coordinates)
def simulate_driver_checkins(num_drivers):
    drivers = []
    for i in range(num_drivers):
        longitude = 77.19  + random.uniform(-0.02, 0.02)  # Random around Delhi longitude
        latitude = 28.54  + random.uniform(-0.02, 0.02)   # Random around Delhi latitude
        driver_id = f"driver{i}"
        redis_client.geoadd("available_drivers", (longitude, latitude, driver_id))
        new_driver = {'driver_id': driver_id, 'longitude': longitude, 'latitude': latitude}
        print(new_driver)
        drivers.append(new_driver)
    return drivers

In [25]:
# Function for simulating rider queries 
def simulate_rider_queries(num_riders):
    riders = []
    for i in range(num_riders):
        rider_id = f"rider{i}"
        user_longitude = 77.19 + random.uniform(-0.02, 0.02)  # Random around Delhi longitude
        user_latitude = 28.54 + random.uniform(-0.02, 0.02)  # Random around Delhi latitude
        radius = RADIUS

        nearby_drivers = redis_client.georadius(
            "available_drivers", user_longitude, user_latitude, radius, unit='km', withdist=True
        )

        # Sort nearby_drivers by distance (ascending order)
        nearby_drivers.sort(key=lambda x: x[1])  # Sort by distance (second element in tuple)

        new_rider = {
            'rider_id': rider_id,
            'longitude': user_longitude,
            'latitude': user_latitude,
            'nearby_drivers': nearby_drivers
        }
        print(new_rider)
        riders.append(new_rider)

    return riders

In [26]:
# Simulate check-ins
drivers = simulate_driver_checkins(NUM_OF_DRIVERS)

{'driver_id': 'driver0', 'longitude': 77.2006203074985, 'latitude': 28.52911423902516}
{'driver_id': 'driver1', 'longitude': 77.1991387495466, 'latitude': 28.53821858388841}
{'driver_id': 'driver2', 'longitude': 77.18245656067984, 'latitude': 28.523506689544405}
{'driver_id': 'driver3', 'longitude': 77.18351983505327, 'latitude': 28.553919311068956}
{'driver_id': 'driver4', 'longitude': 77.1905731564784, 'latitude': 28.526580808362475}
{'driver_id': 'driver5', 'longitude': 77.20926102511525, 'latitude': 28.559446731610844}
{'driver_id': 'driver6', 'longitude': 77.20030407594389, 'latitude': 28.547075308520988}
{'driver_id': 'driver7', 'longitude': 77.2024447595539, 'latitude': 28.533051571443583}
{'driver_id': 'driver8', 'longitude': 77.19793214027027, 'latitude': 28.549358799468944}
{'driver_id': 'driver9', 'longitude': 77.18883226678183, 'latitude': 28.536713078381837}


In [27]:
# Simulate rider queries
riders = simulate_rider_queries(NUM_OF_RIDERS)


{'rider_id': 'rider0', 'longitude': 77.20801635265057, 'latitude': 28.557640185927745, 'nearby_drivers': [[b'driver5', 0.2349]]}
{'rider_id': 'rider1', 'longitude': 77.18476921871225, 'latitude': 28.549560297899994, 'nearby_drivers': [[b'driver3', 0.5]]}
{'rider_id': 'rider2', 'longitude': 77.18804412064091, 'latitude': 28.543499621940192, 'nearby_drivers': [[b'driver9', 0.7587]]}
{'rider_id': 'rider3', 'longitude': 77.18915225682217, 'latitude': 28.543412472443443, 'nearby_drivers': [[b'driver9', 0.7458]]}
{'rider_id': 'rider4', 'longitude': 77.20289519761985, 'latitude': 28.552427932094133, 'nearby_drivers': [[b'driver8', 0.593], [b'driver6', 0.647], [b'driver5', 0.9982]]}
{'rider_id': 'rider5', 'longitude': 77.17868223284201, 'latitude': 28.556618360634538, 'nearby_drivers': [[b'driver3', 0.5597]]}
{'rider_id': 'rider6', 'longitude': 77.19281344150588, 'latitude': 28.52925528275391, 'nearby_drivers': [[b'driver4', 0.3693], [b'driver0', 0.763], [b'driver9', 0.9162]]}
{'rider_id': 'ri

In [28]:
df_drivers = gpd.GeoDataFrame(drivers, geometry=[Point(row['longitude'], row['latitude']) for row in drivers])
df_riders = gpd.GeoDataFrame(riders, geometry=[Point(row['longitude'], row['latitude']) for row in riders])

In [29]:
# Create a map
center_location = (df_drivers.geometry.y.mean(), df_drivers.geometry.x.mean())  # Average location
map = folium.Map(location=center_location, zoom_start=15)

map

In [30]:
map = folium.Map(location=center_location, zoom_start=15, tiles="CartoDB Positron")

map

In [31]:
# Add driver markers with Taxi emoji 🚕
for _, row in df_drivers.iterrows():
    driver_emoji = "🚕"
    driver_id = row["driver_id"]
    folium.Marker(location=[row.geometry.y, row.geometry.x], icon=folium.DivIcon(
        icon_size=(25,25),
        icon_anchor=(0,0),
        html=f"""
            <div style="font-size: 30px;">{driver_emoji}</div>
            <div style="font-size: 15px; font-weight: bold;">{driver_id}</div>
        """
    )
        ).add_to(map)


# Add rider markers with 🙋‍♂️ or 🙋‍♀️ 
for _, row in df_riders.iterrows():
    # because diversity is important
    rider_emoji = random.choice(["🙋‍♂️", "🙋‍♀️"])
    rider_id = row["rider_id"] 

    folium.Marker(location=[row.geometry.y, row.geometry.x],icon=folium.DivIcon(
        icon_size=(25,25),
        icon_anchor=(0,0),
       html=f"""
            <div style="font-size: 30px;">{rider_emoji}</div>
            <div style="font-size: 15px; font-weight: bold;">{rider_id}</div>
        """
    )
        ).add_to(map)
map.fit_bounds(map.get_bounds())

map

In [32]:
def draw_match_line(map, rider_location, driver_location):
    # Extract coordinates from Point objects (if needed)
    if isinstance(rider_location, Point):
        rider_coords = [rider_location.y, rider_location.x]
    else:
        rider_coords = rider_location

    driver_coords = driver_location

    # Calculate distance (in kilometers)
    distance = geopy.distance.distance(rider_coords, driver_coords).km

    # Create polyline with distance label
    folium.PolyLine(
        locations=[rider_coords, driver_coords], color="green", weight=4, opacity=1, dashArray="5, 5"
    ).add_to(map)


In [33]:
# Simulation time!
# In an ideal world when nearest driver always accept your ride.
# Match riders to drivers and draw lines, with logs
matched_drivers = set()

for i, rider in df_riders.iterrows():
    rider_id = rider["rider_id"]
    print("-"*10)
    print(f"Time: t{i}")
    print("-"*10)
    print(f"{rider_id} looking for a ride...")

    nearby_drivers = rider["nearby_drivers"]
    nearby_drivers.sort(key=lambda x: x[1])

    print(f"Drivers found nearby: {nearby_drivers}")

    found_driver = False
    for driver_info, distance in nearby_drivers:
        driver_id = driver_info.decode()
        if driver_id not in matched_drivers:
            matched_drivers.add(driver_id)
            found_driver = True
            print(f"Ride matched to: {driver_id}")
            break

    # If no available driver is found, draw a red circle around the rider
    if not found_driver:
        print(f"No driver found for {rider_id}!")
        folium.Circle(
            location=[rider.geometry.y, rider.geometry.x],
            radius=RADIUS*1000,  # Adjust radius as needed
            color="red",
            fill=True,
            fill_color="red",
            fill_opacity=0.1,
        ).add_to(map)
        continue

    if driver_id in matched_drivers:
        driver_row = df_drivers[df_drivers["driver_id"] == driver_id].iloc[0]
        driver_location = [driver_row.geometry.y, driver_row.geometry.x]  # Extract coordinates
        draw_match_line(map, rider.geometry, driver_location)
        print("Ride is On!\n")

# Display the map with lines
map

----------
Time: t0
----------
rider0 looking for a ride...
Drivers found nearby: [[b'driver5', 0.2349]]
Ride matched to: driver5
Ride is On!

----------
Time: t1
----------
rider1 looking for a ride...
Drivers found nearby: [[b'driver3', 0.5]]
Ride matched to: driver3
Ride is On!

----------
Time: t2
----------
rider2 looking for a ride...
Drivers found nearby: [[b'driver9', 0.7587]]
Ride matched to: driver9
Ride is On!

----------
Time: t3
----------
rider3 looking for a ride...
Drivers found nearby: [[b'driver9', 0.7458]]
No driver found for rider3!
----------
Time: t4
----------
rider4 looking for a ride...
Drivers found nearby: [[b'driver8', 0.593], [b'driver6', 0.647], [b'driver5', 0.9982]]
Ride matched to: driver8
Ride is On!

----------
Time: t5
----------
rider5 looking for a ride...
Drivers found nearby: [[b'driver3', 0.5597]]
No driver found for rider5!
----------
Time: t6
----------
rider6 looking for a ride...
Drivers found nearby: [[b'driver4', 0.3693], [b'driver0', 0.763